Extracting User's Google Calendar

In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import re
import ollama
import pandas as pd
from ollama import Client
from ollama import chat
import methods
import formats
import json



google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() 
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar)
now = methods.current_time()

No tasks found.


Logging Previous Conversation

In [4]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [5]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

In [6]:
def log_activity(role, message):
    filename = f"activity_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

Get User's Query

In [7]:
past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and To-Do list for the week, store it for your reference. Calendar: {calendar}\n To-Do Tasks: {tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my virtual assistant, you are an expert with managing the calendar and scheduling tasks. 
First, you can create, move, check, list, update, delete, and handle recurring events based on the user's requests.
When the user doesn't otherwise provide enough information, you can ask for more details, upto 4 times. Otherwise use your best judgement.
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
Check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions.
Second, you can help me manage my to-do list by using your own judgement and scheduling the things on it onto my calendar.
When the user wants to schedule their to-do tasks, reorder the tasks by priority using your judgement and output all the tasks with proposed timeslots for each task. Create a balanced schedule.
Check if a task is already in the calendar. 
Take your time to think and plan the schedule before presenting it to the user.
Also use your best guess to determine the length of the tasks. Assume that all the tasks have to be scheduled.
If the user agrees, then add the events to the calendar and let the user know that the tasks have been scheduled.
If the user doesn't agree, then regenerate the timeslots for the tasks and ask for confirmation again. 
When you are done, let the user know that you have completed your task. Unless you are done with the task, always end with a question mark.
Please perform the necessary actions using the Google Calendar API.\n
Example input: 'schedule all my tasks'
Example output: 'Sure! Here's a proposed schedule for all your tasks. Is there anything you'd like to change?
- Task 1: Apr 2nd, 10:00 AM - 11:00 AM
- Task 2: Apr 2nd, 11:00 AM - 12:00 PM
- Task 3: Apr 2nd, 12:00 PM - 1:00 PM\n
Here's the previous conversation with the user, read it and understand user preferences if any, use it to make your best judgement on scheduling.\n"""

# initial setup

initial_prompt = context + instruction + past_conversation

In [8]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


can you schedule a meeting for me at 5pm tomorrow


In [9]:
def llama2(client, messages):
    response = ollama.chat(model='llama2', messages=messages)
    return response['message']['content']

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]
log_message("user", user_request)

In [10]:
ollama.pull("llama2")

{'status': 'success'}

In [11]:
finalized_response = False
client = Client(host='http://localhost:11434')
#establishing feedback loop to chat with user
while not finalized_response:
    response = llama2(client, messages)
    print(response)
    log_message("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
    else:
        finalized_response = True


Of course! I'd be happy to help you schedule a meeting. Can you please provide me with more details such as the purpose of the meeting, the duration, and any specific time slots that you prefer? This will help me create a proposed schedule that fits your needs.
1 hour duration
Great, thank you for letting me know! Based on your request, I have checked your calendar and found a available time slot at 5:00 PM tomorrow. Is this time suitable for the meeting? If not, please let me know any other time slots that you prefer.
yes
Excellent! I have scheduled the meeting for 5:00 PM tomorrow. Please confirm if there are any changes or cancellations.

Is there anything else I can help you with?
no
Great, thank you for letting me know! I have successfully scheduled the meeting at 5:00 PM tomorrow. If you need any further assistance, please don't hesitate to ask. Have a great day!


In [15]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, assume default duration as 1 hour for any event. Use createEvent for scheduling tasks on the to-do list.
Available function templates to include in the response string:
- createEvent(description, date, time, duration)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_color, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, start_date, time, duration, frequency, until_date). Frequency can have the value of DAILY, WEEKLY, MONTHLY, YEARLY.
- deleteRecurringEvent(recurring_event_id) 
- updateRecurringEvent(recurring_event_id, new_color, new_duration, new_location, new_frequency, new_until)
For all functions other than createEvent and createRecurringEvent, make sure to fill in the id of the event you are referring to from the calendar provided.
Make sure to fill in the required information, such as event names, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.
Example input1: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output1: 'createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1:00)'
Example input2: 'Update my team meeting to be two hours'
Example output2: 'updateEvent("event_id", '', 2:00, '')
Example input3: 'Create a recurring meeting every Wednesday at 2pm until the end of June'
Example output3: 'createRecurringEvent("Meeting", "YYYY-MM-DD", "02:00", "1:00", "WEEKLY", "YYYY-MM-DD")
Example input4: 'Update my recurring meeting every Wednesday at 2pm to be every month until the end of August'
Example output4: 'updateRecurringEvent("RecurringEventId", "", "02:00", "1:00", "", "MONTHLY", "YYYY-MM-DD")
Example input5: 'Move my team meeting to tomorrow at 12pm'
Example output5: 'moveEvent("event_id", 'YYYY-MM-DD', '12:00')'
Example input6: 'Clear my afternoon'
Example output7: 'deleteEvent("event_id");deleteEvent("event_id")'
Example input8: 'Schedule all my tasks'
Example output8: 'createEvent("Task 1", "YYYY-MM-DD", "10:00", 1:00);createEvent("Task 2", "YYYY-MM-DD", "11:00", 1:00);createEvent("Task 3", "YYYY-MM-DD", "12:00", 1:00)\n

YYYY-MM-DD replace it with the date user want

Now, please generate the necessary function call(s) based on the user's request."""

In [16]:
messages.append({
    "role": "system",
    "content": instruction2
})

In [17]:
log_activity("system", instruction2)
function_call = llama2(client, messages)
log_activity("assistant", function_call)
print(function_call)


Sure! Based on the user's request, here are the appropriate function calls to interact with the Google Calendar API:

createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1:00);
updateEvent("event_id", '', 2:00, '');
createRecurringEvent("Meeting", "YYYY-MM-DD", "02:00", "1:00", "WEEKLY", "YYYY-MM-DD");
updateRecurringEvent("RecurringEventId", "", "02:00", "1:00", "", "MONTHLY", "YYYY-MM-DD");
moveEvent("event_id", 'YYYY-MM-DD', '12:00');
deleteEvent("event_id");deleteEvent("event_id");
createEvent("Task 1", "YYYY-MM-DD", "10:00", 1:00);createEvent("Task 2", "YYYY-MM-DD", "11:00", 1:00);createEvent("Task 3", "YYYY-MM-DD", "12:00", 1:00)

Note that the `YYYY-MM-DD` format is used for dates, and the `YYYY-MM-DD` format is used for recurring events. Also, make sure to fill in the required information based on the user's request context.


In [13]:
function_calls = [line.strip() for line in function_call.split(';')]
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    if call == '': continue
    parts = call.split('(')
    function_name = parts[0]

    arguments = parts[1].rstrip(')').split(', ')
    

    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['Sure! Based on your input, I suggest scheduling the jogging event for tomorrow at 12pm and moving the team meeting to tomorrow at 12pm. Here are the appropriate function calls:\n\ncreateEvent("Jogging", "YYYY-MM-DD", "12:00", 1:00)', 'moveEvent("event_id", \'YYYY-MM-DD\', \'12:00\')', '']
Function 'Sure! Based on your input, I suggest scheduling the jogging event for tomorrow at 12pm and moving the team meeting to tomorrow at 12pm. Here are the appropriate function calls:

createEvent' not found.
Error: <HttpError 404 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events/event_id?alt=json returned "Not Found". Details: "[{'domain': 'global', 'reason': 'notFound', 'message': 'Not Found'}]">
